In [13]:
import pandas as pd
import numpy as np

class Problema:
    def __init__(self, arquivo_distancia, arquivo_tempo):
        self.distancia = pd.read_csv(arquivo_distancia, header=None).to_numpy(dtype=float)
        self.tempo     = pd.read_csv(arquivo_tempo,     header=None).to_numpy(dtype=float)
        self.n = self.distancia.shape[0]  # número de cidades

    def custo_arco(self, i, j, objetivo):
        # objetivo: "distancia" ou "tempo"
        if objetivo == "distancia":
            return self.distancia[i, j]
        else:
            return self.tempo[i, j]

def custo_rota(problema, rota, objetivo):
    # Soma o custo ao longo da rota circular (0 -> ... -> 0)
    c = 0.0
    for k in range(len(rota) - 1):
        c += problema.custo_arco(rota[k], rota[k+1], objetivo)
    c += problema.custo_arco(rota[-1], rota[0], objetivo)  # volta à origem
    return c

def construtiva_vizinho_mais_proximo(problema, objetivo):
    # Gera uma rota inicial simples: vizinho mais próximo a partir da cidade 0.
    n = problema.n
    rota = [0]
    visitado = [False] * n
    visitado[0] = True
    atual = 0

    for cidade in range(n - 1):  # repetimos n-1 escolhas 
        proxima_cidade = None
        menor_custo = float("inf")

        for j in range(n):
            if not visitado[j]:
                custo = problema.custo_arco(atual, j, objetivo)
                if custo < menor_custo:
                    menor_custo = custo
                    proxima_cidade = j

        rota.append(proxima_cidade)
        visitado[proxima_cidade] = True
        atual = proxima_cidade

    return rota

def main():
    problema = Problema("distancia.csv", "tempo.csv")

    # escolha qual função objetivo otimizar nesta execução
    objetivo = "distancia"  # "distancia" ou "tempo"

    # rota inicial (vizinho mais próximo)
    rota = construtiva_vizinho_mais_proximo(problema, objetivo)

    # custo da rota
    c = custo_rota(problema, rota, objetivo)

    print(f"n = {problema.n}")
    print(f"Objetivo: {objetivo}")
    print(f"Custo da rota inicial: {c:.3f}")
    print(f"Rota (primeiros 10 nós): {rota[:10]}")

main()


n = 250
Objetivo: distancia
Custo da rota inicial: 1586.700
Rota (primeiros 10 nós): [0, 72, 214, 248, 33, 120, 22, 11, 169, 51]


In [14]:
# definição das vizinhanças 

# K1: SWAP (2-exchange) — troca as posições a e b
def vizinhanca_swap(rota, a, b):
    # não mexe na posição 0 e ignora casos triviais
    if a == 0 or b == 0 or a == b:
        return rota
    n = len(rota)
    if not (0 <= a < n and 0 <= b < n):
        return rota
    nova = rota.copy()
    nova[a], nova[b] = nova[b], nova[a]
    return nova

# K2: OR-OPT (k=1) — remove a cidade em a e reinsere antes da posição b
def vizinhanca_oropt1(rota, a, b):
    # não mexe na posição 0; evita b == a ou b == a+1 (inserção nula)
    if a == 0 or b == 0 or b == a or b == a + 1:
        return rota
    n = len(rota)
    if not (0 <= a < n and 0 <= b <= n):  # b pode ser n (inserir no fim antes de voltar ao 0)
        return rota
    nova = rota.copy()
    cidade = nova.pop(a)
    if b > a:
        b -= 1  # corrige deslocamento do índice após o pop
    nova.insert(b, cidade)
    return nova

# K3: 2-OPT (dirigido) — reverte o segmento [a, b]
def vizinhanca_2opt(rota, a, b):
    # mantém a origem na posição 0; exige 1 <= a < b <= n-2 para preservar a ligação final com 0
    n = len(rota)
    if a <= 0 or b >= n - 1 or a >= b:
        return rota
    nova = rota.copy()
    nova[a:b+1] = reversed(nova[a:b+1])
    return nova

# aplica a vizinhança K em (a, b)
# K=1 -> swap, K=2 -> or-opt(1), K=3 -> 2-opt
def aplicar_vizinhanca(rota, k, a, b):
    if k == 1:
        return vizinhanca_swap(rota, a, b)
    if k == 2:
        return vizinhanca_oropt1(rota, a, b)
    if k == 3:
        return vizinhanca_2opt(rota, a, b)

    return rota


In [15]:
# busca local

def diferenca_custo_k1(problema, rota, a, b, objetivo):
    # K1: swap a<b 
    n = len(rota)
    if a == 0 or b == 0 or a >= b:
        return 0.0
    A_prev = rota[a-1]; A = rota[a]; A_next = rota[a+1] if a+1 < n else rota[0]
    B_prev = rota[b-1]; B = rota[b]; B_next = rota[b+1] if b+1 < n else rota[0]
    antigo = (problema.custo_arco(A_prev, A, objetivo) +
              problema.custo_arco(A, A_next, objetivo) +
              problema.custo_arco(B_prev, B, objetivo) +
              problema.custo_arco(B, B_next, objetivo))
    if b == a + 1:
        novo = (problema.custo_arco(A_prev, B, objetivo) +
                problema.custo_arco(B, A, objetivo) +
                problema.custo_arco(A, B_next, objetivo))
    else:
        novo = (problema.custo_arco(A_prev, B, objetivo) +
                problema.custo_arco(B, A_next, objetivo) +
                problema.custo_arco(B_prev, A, objetivo) +
                problema.custo_arco(A, B_next, objetivo))
    return novo - antigo

def diferenca_custo_k2(problema, rota, a, b, objetivo):
    # K2: or-opt(1) remove em a e reinsere antes de b
    n = len(rota)
    if a == 0 or b == 0 or b == a or b == a + 1:
        return 0.0
    pred_a = rota[a-1]; a_i = rota[a]; suc_a = rota[a+1] if a+1 < n else rota[0]
    pred_b = rota[b-1]; b_i = rota[b] if b < n else rota[0]
    antigo = (problema.custo_arco(pred_a, a_i, objetivo) +
              problema.custo_arco(a_i, suc_a, objetivo) +
              problema.custo_arco(pred_b, b_i, objetivo))
    novo = (problema.custo_arco(pred_a, suc_a, objetivo) +
            problema.custo_arco(pred_b, a_i, objetivo) +
            problema.custo_arco(a_i, b_i, objetivo))
    return novo - antigo

def diferenca_custo_k3(problema, rota, a, b, objetivo):
    # K3: 2-opt reverte [a,b]
    n = len(rota)
    if a <= 0 or b >= n - 1 or a >= b:
        return 0.0
    i = rota[a-1]; j = rota[a]; k = rota[b]; l = rota[b+1]
    antigo = problema.custo_arco(i, j, objetivo) + problema.custo_arco(k, l, objetivo)
    novo   = problema.custo_arco(i, k, objetivo) + problema.custo_arco(j, l, objetivo)
    return novo - antigo

# first improvement dentro da vizinhança K
def first_improvement(problema, rota, objetivo, k):
    # retorna (rota_candidata, custo_candidato)
    n = len(rota)
    custo_atual = custo_rota(problema, rota, objetivo)

    if k == 1:
        for a in range(1, n-1):
            for b in range(a+1, n):
                d = diferenca_custo_k1(problema, rota, a, b, objetivo)
                if d < 0.0:
                    nova = vizinhanca_swap(rota, a, b)
                    return nova, custo_atual + d

    elif k == 2:
        for a in range(1, n-1):
            for b in range(1, n):
                if b == a or b == a + 1:
                    continue
                d = diferenca_custo_k2(problema, rota, a, b, objetivo)
                if d < 0.0:
                    nova = vizinhanca_oropt1(rota, a, b)
                    return nova, custo_atual + d

    elif k == 3:
        for a in range(1, n-2):
            for b in range(a+1, n-1):
                d = diferenca_custo_k3(problema, rota, a, b, objetivo)
                if d < 0.0:
                    nova = vizinhanca_2opt(rota, a, b)
                    return nova, custo_atual + d

    return rota, custo_atual


def neighborhood_change(rota, custo, rota_cand, custo_cand, k):
    if custo_cand < custo:
        return rota_cand, custo_cand, 1
    else:
        return rota, custo, k + 1

def vnd(problema, rota_inicial, objetivo, kmax=3):
    rota = rota_inicial[:]
    custo = custo_rota(problema, rota, objetivo)
    k = 1
    while k <= kmax:
        rota_cand, custo_cand = first_improvement(problema, rota, objetivo, k)
        rota, custo, k = neighborhood_change(rota, custo, rota_cand, custo_cand, k)
    return rota, custo


In [18]:
# carrega o problema
problema = Problema("distancia.csv", "tempo.csv")

# escolha do objetivo: "distancia" ou "tempo"
objetivo = "distancia"

# constrói rota inicial e calcula custo
rota0 = construtiva_vizinho_mais_proximo(problema, objetivo)
c0 = custo_rota(problema, rota0, objetivo)
print("custo inicial:", c0)

# roda a busca local VND (K=1 -> K=3)
rota1, c1 = vnd(problema, rota0, objetivo, kmax=3)
print("custo após VND:", c1)

custo inicial: 1586.7000000000016
custo após VND: 1556.0
